1. read .las pointcloud file
2. convert the pointcloud to the reference local coordinates
3. read bounding boxes
5. enlarge bounding boxes
6. crop points within enlarged bounding boxes
7. write cropped pointcloud objects

In [1]:
"""# google colab installation
!pip install open3d
!pip install laspy
!pip install pptk
"""

'# google colab installation\n!pip install open3d\n!pip install laspy\n!pip install pptk\n'

In [2]:
"""# for google colab 
from google.colab import drive
import laspy
import numpy as np
import os
import open3d as o3d

drive.mount("/content/drive")
os.chdir('/content/drive/MyDrive/DREAMS - Zhiang/Projects/3D_rock_detection/data')
print(os.listdir())
"""

'# for google colab \nfrom google.colab import drive\nimport laspy\nimport numpy as np\nimport os\nimport open3d as o3d\n\ndrive.mount("/content/drive")\nos.chdir(\'/content/drive/MyDrive/DREAMS - Zhiang/Projects/3D_rock_detection/data\')\nprint(os.listdir())\n'

In [ ]:
import laspy
import numpy as np
import os
import open3d as o3d

os.chdir('/content/drive/MyDrive/DREAMS - Zhiang/Projects/3D_rock_detection/data')
print(os.listdir())

# read pointcloud .las

In [ ]:
pc = laspy.read('granite_dells_wgs_utm.las')

In [ ]:
# WGS 84 & UTM 12N
print(pc.x.scaled_array().min())
print(pc.x.scaled_array().max())
print(pc.y.scaled_array().min())
print(pc.y.scaled_array().max())
print(pc.z.scaled_array().min())
print(pc.z.scaled_array().max())

# color value has type of uint16
print(pc.red.max())
print(pc.red.min())
print(pc.green.max())
print(pc.green.min())
print(pc.blue.max())
print(pc.blue.min())

369880.600594
369970.394791
3829104.674131
3829197.061813
-5.784746
25.580458
65280
0
65280
0
65280
0


# read bounding box

In [ ]:
bboxes = np.load('pbr_bboxes_wgs_utm.npy')

x1,y1,x2,y2 = bboxes[0]
print(x1,x2,y1,y2)

369920.6882544258 369924.83291339944 3829185.267451371 3829188.154320308


In [ ]:
def box_filter(las, x1, y1, x2, y2, padding=0.2):
  x1 = x1 - padding
  x2 = x2 + padding
  y1 = y1 - padding
  y2 = y2 + padding
  xgood = (las.x >= x1) & (las.x < x2)
  ygood = (las.y >= y1) & (las.y < y2)
  good = xgood & ygood 
  found = (las.x.scaled_array()[good], las.y.scaled_array()[good], las.z.scaled_array()[good], las.red[good], las.green[good], las.blue[good])
  return found

In [ ]:
def write_las_file(f, pt):
  header = laspy.LasHeader(point_format=2, version="1.2")
  las = laspy.LasData(header)
  las.x = pt[0]
  las.y = pt[1]
  las.z = pt[2]
  las.red = pt[3]
  las.green = pt[4]
  las.blue = pt[5]
  las.write('box_pbr/'+f)

In [ ]:
for id, bbox in enumerate(bboxes):
  x1,y1,x2,y2 = bbox
  pbr_pc = box_filter(pc, x1, y1, x2, y2)
  write_las_file('pbr{i}.las'.format(i=id), pbr_pc)
  print(id)

## Convert .las to .pcd

In [3]:
las_files = ['box_pbr/' + f for f in os.listdir('box_pbr/') if f.endswith('.las')]

In [5]:
for las_file in las_files[:5]:
  pc = laspy.read(las_file)
  x = pc.x.scaled_array()
  y = pc.y.scaled_array()
  z = pc.z.scaled_array()
  r = np.uint8(pc.red/65535.*255)
  g = np.uint8(pc.green/65535.*255)
  b = np.uint8(pc.blue/65535.*255)
  rgb = np.vstack((r, g, b)).transpose()
  xyz = np.vstack((x, y, z)).transpose()


In [12]:

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd.colors = o3d.utility.Vector3dVector(rgb)

In [13]:
o3d.io.write_point_cloud("pbr.pcd", pcd)

True